### Compiling with the mqtt_as library as frozen bytecode

Instructions from:
    https://www.esp8266.com/wiki/doku.php?id=toolchain

Make the esp C-compiler (done already)

```
conda deactivate
git clone --recursive https://github.com/pfalcon/esp-open-sdk
cd esp-open-sdk
make
```

Clone and submodule update the base micropython build
```
git clone git@github.com:goatchurchprime/micropython.git micropython
cd micropython
git submodule update --init
```


Copy across the modules (from the extrepositories directory)
```
cp micropython-mqtt/mqtt_as/mqtt_as.py micropython/ports/esp8266/modules/
cp micropython-mqtt/mqtt_as/mqtt_as_timeout.py micropython/ports/esp8266/modules/

cp -r micropython-lib/umqtt.simple/umqtt/ micropython/ports/esp8266/modules/
cp micropython-lib/umqtt.simple/umqtt/simple.py micropython/ports/esp8266/modules/umqtt/
cp micropython-lib/umqtt.robust/umqtt/robust.py micropython/ports/esp8266/modules/umqtt/
cp /home/julian/repositories/jupyter_micropython_developer_notebooks/async_web_serve/lighthttpserve_funcs.py micropython/ports/esp8266/modules/

export PATH="/home/julian/extrepositories/esp-open-sdk/xtensa-lx106-elf/bin/:$PATH"
```

Build the cross compiler
```
cd micropython/mpy-cross
make
```

Make the file
```
cd micropython/ports/esp8266
make -j4
```

Result will be in:
> micropython/ports/esp8266/build-GENERIC/firmware-combined.bin


In [1]:
%esptool erase

Executing:
  esptool.py --port /dev/ttyUSB3 erase_flash

esptool.py v2.8
Serial port /dev/ttyUSB3
Connecting....
[Press the PRG button now if required]
Detecting chip type... ESP8266
Chip is ESP8266EX
Features: WiFi
Crystal is 26MHz
MAC: 3c:71:bf:2c:af:0f
Uploading stub...
Running stub...
Stub running...
Erasing flash (this may take a while)...
Chip erase completed successfully in 7.4s
Hard resetting via RTS pin...


In [1]:
%esptool esp8266 /home/julian/extrepositories/micropython/ports/esp8266/build-GENERIC/firmware-combined.bin

Executing:
  esptool.py --port /dev/ttyUSB0 --baud 460800 write_flash --flash_size=detect -fm dio 0 /home/julian/extrepositories/micropython/ports/esp8266/build-GENERIC/firmware-combined.bin

esptool.py v2.8
Serial port /dev/ttyUSB0
Connecting....
[Press the PRG button now if required]
Detecting chip type... ESP8266
Chip is ESP8266EX
Features: WiFi
Crystal is 26MHz
MAC: 98:f4:ab:da:58:01
Uploading stub...
Running stub...
Stub running...
Changing baud rate to 460800
Changed.
Configuring flash size...
Auto-detected Flash size: 4MB
Flash params set to 0x0240
Compressed 640052 bytes to 420021...
Wrote 640052 bytes (420021 compressed) at 0x00000000 in 9.6 seconds (effective 532.3 kbit/s)...
Hash of data verified.

Leaving...
Hard resetting via RTS pin...


In [2]:
%serialconnect
help("modules")

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.
__main__          lwip              uasyncio/funcs    upip_utarfile
_boot             machine           uasyncio/lock     urandom
_onewire          math              uasyncio/stream   ure
_uasyncio         micropython       ubinascii         uselect
_webrepl          mqtt_as           ucollections      usocket
apa102            mqtt_as_timeout   ucryptolib        ussl
btree             neopixel          uctypes           ustruct
builtins          network           uerrno            utime
dht               ntptime           uhashlib          utimeq
ds18x20           onewire           uheapq            uwebsocket
esp               port_diag         uio               uzlib
flashbdev         sys               ujson             webrepl
framebuf          uarray            umqtt/robust      webrepl_setup
gc                uasyncio/__init__ umqtt/simple      websocket_helper
inisetup          uasyncio/core     uos
lighthttpserve_funcs    

In [3]:
import esp
help(esp)

object <module 'esp'> is of type module
  __name__ -- esp
  osdebug -- <function>
  sleep_type -- <function>
  deepsleep -- <function>
  flash_id -- <function>
  flash_read -- <function>
  flash_write -- <function>
  flash_erase -- <function>
  flash_size -- <function>
  flash_user_start -- <function>
  neopixel_write -- <function>
  apa102_write -- <function>
  dht_readinto -- <function>
  freemem -- <function>
  meminfo -- <function>
  check_fw -- <function>
  info -- <function>
  malloc -- <function>
  free -- <function>
  esf_free_bufs -- <function>
  set_native_code_location -- <function>
  SLEEP_NONE -- 0
  SLEEP_LIGHT -- 1
  SLEEP_MODEM -- 2


In [39]:
%sendtofile main.py

from machine import Pin
from esp import my9291_write
DI = Pin(13, Pin.OUT) 
DCK = Pin(15, Pin.OUT)
my9291_write(DI, DCK, b'\x00')

Sent 5 lines (129 bytes) to main.py.


In [40]:
%sendtofile --append main.py

import time
while 1:
    my9291_write(DI, DCK, bytes([0,0,0,10,0,0,0,0]))
    time.sleep_ms(100)
    my9291_write(DI, DCK, bytes([0,10,0,0,0,0,0,0]))
    time.sleep_ms(100)
    my9291_write(DI, DCK, bytes([0,0,0,0,0,10,0,0]))
    time.sleep_ms(100)
    my9291_write(DI, DCK, bytes([0,0,0,0,0,0,0,5]))
    time.sleep_ms(100)
    my9291_write(DI, DCK, bytes([0,1,0,1,0,1,0,1]))
    time.sleep_ms(100)


Sent 12 lines (399 bytes) to main.py.


In [43]:
import main




*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
  File "main.py", line 11, in <module>
KeyboardInterrupt: 


In [4]:
# Small ESP8266

#      RST          |  GPIO1   TX
#       A0          |  GPIO3   RX
#       D0  GPIO16  |  GPIO5   D1  SCL
# SCK   D5  GPIO14  |  GPIO4   D2  SDA
# MISO  D6  GPIO12  |  GPIO0   D3
# MOSI  D7  GPIO13  |  GPIO2   D4  LED
# SS    D8  GPIO15  |          GND
#      3V3          |          5V

# (not able to get the mDNS to work for esp8266)

No serial connected
  %serialconnect to connect
  %esptool to flash the device
  %lsmagic to list commands

In [44]:
# Setting up the webrepl hotspot with password=wpass
%serialconnect --raw
%readbytes 

# Print out the hotspot name (while we are here)
%writebytes "import network, ubinascii\r\n"
%readbytes
%writebytes "\r\n"
%readbytes
%writebytes "'MicroPython-%s'%ubinascii.hexlify(network.WLAN().config('mac')[-3:]).decode()\r\n"
%comment "This is the ESP8266's hotspot name"
%readbytes
%writebytes "\r\n"
%readbytes

# soft reset so import works
%writebytes "\x04"
%readbytes
%writebytes "import webrepl_setup\r\n"
%readbytes 
%writebytes "E\r\n"
%readbytes

# uncomment if second time (with additional question of changing password)
#%writebytes "y\r\n"
#%readbytes

# set the password
%writebytes "wpass\r\n"
%readbytes
%writebytes "wpass\r\n"
%readbytes

#%writebytes "y\r\n"
#%readbytes

%writebytes "\x04"
%readbytes
%disconnect --raw

Connecting to --port=/dev/ttyUSB2 --baud=115200 
import network, ubinascii
>>> 
>>> This is the ESP8266's hotspot name'MicroPython-%s'%ubinascii.hexlify(network.WLAN().config('mac')[-3:]).decode()
'MicroPython-7ec9f2'
>>> 
>>> 
MPY: soft reboot

Closing serial Serial<id=0x7f98baf1ba90, open=True>(port='/dev/ttyUSB2', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)


In [40]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB4 --baud=115200 
Ready.


In [2]:
%sendtofile --source ../async_web_serve/lighthttpserve_funcs.py

Sent 230 lines (7009 bytes) to lighthttpserve_funcs.py.


In [41]:
from machine import Pin, PWM
p = Pin(15, Pin.OUT)
x = PWM(p)
x.freq(3)
x.duty(10)
#p.value(1)

In [34]:
x = PWM(p)
x.freq(20)
x.duty(10)

In [29]:
print(x, p.value())

PWM(2, freq=2, duty=10) 0


In [16]:

x.value(0)

Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
AttributeError: 'PWM' object has no attribute 'value'


In [1]:
%serialconnect

Connecting to --port=/dev/ttyUSB4 --baud=115200 
Ready.


In [2]:
%sendtofile --source utils.py

Sent 31 lines (960 bytes) to utils.py.


In [66]:
from machine import Pin
import time
p = Pin(14, Pin.OUT)
for i in range(10):
    p.value(i%2)
    time.sleep(0.2)

.

In [67]:
p = Pin(12, Pin.IN, Pin.PULL_UP)

In [68]:
for i in range(20):
    print(p.value())
    time.sleep(1)

1
1
1
1
1
1
0
0
.0
1
1
1
1
1
1
1
1
1
.1
1
